## Inteligencia Artificial: Evaluación 1

### Hecho por:
* David Daniel Henriquez Leal
* Santiago Andres Mercado Barandica

### Librerias utilizadas en el programa

In [ ]:
import pyautogui
import cv2
import numpy as np
import easyocr
import matplotlib.pyplot as plt
from collections import defaultdict

### Algoritmos de búsqueda de la jugada segura

Este código encuentra una jugada segura en el buscaminas utilizando una combinación de técnicas: marcado de minas evidentes, identificación de movimientos seguros directos, y un enfoque BT-FC-DO para calcular el riesgo de las celdas desconocidas. 
Primero marca internamente las minas obvias cuando hay casillas con mismo número de casillas ocultas que minas adyacentes.
En base a esta información, busca movimientos seguros obvios, es decir si ya marcó todas las minas adyacentes y aún hay casillas desconocidas.
Si no encuentra una jugada segura con esta heurística, utiliza el algoritmo de BT-FC-DO.
Con este enfoque genera múltiples configuraciones posibles del tablero, calculando la probabilidad de que cada celda sea una mina. 
Finalmente, selecciona una celda con riesgo 0 de mina como una jugada segura y la muestra al usuario.

In [ ]:
#Algoritmo para hallar jugada segura dada la matriz del tablero

def get_neighbors(x, y, board): #Función para obtener los vecinos de una celda
    neighbors = []
    rows, cols = board.shape
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                neighbors.append((nx, ny))
    return neighbors

def mark_mines(board): #Función para marcar las minas evidentes facilitando el analisis
    rows, cols = board.shape
    mines = set()

    for x in range(rows):
        for y in range(cols):
            if board[x, y] > 0:
                unknown_neighbors = []
                for nx, ny in get_neighbors(x, y, board):
                    if board[nx, ny] == 0:
                        unknown_neighbors.append((nx, ny))

                if len(unknown_neighbors) == board[x, y]: #Si el número de vecinos desconocidos es igual al número de minas adyacentes
                    for pos in unknown_neighbors:         #Se marcan como minas las celdas adyacentes
                        mines.add(pos)
    
    #print("Mines:", mines)
    return mines

def safe_moves(board, mines): #Función para encontrar los movimientos seguros evidentes
    rows, cols = board.shape  
    safe_moves = set()

    for x in range(rows):
        for y in range(cols):
            if board[x, y] > 0:
                unknown_neighbors = []
                flagged_neighbors = 0
                for nx, ny in get_neighbors(x, y, board):
                    if board[nx, ny] == 0 and (nx, ny) not in mines:
                        unknown_neighbors.append((nx, ny))
                    elif (nx, ny) in mines:
                        flagged_neighbors += 1
                # Si el número de vecinos desconocidos es mayor al número de minas adyacentes y ya se han marcado todas las minas
                if len(unknown_neighbors) >  board[x, y] - flagged_neighbors and board[x, y] == flagged_neighbors:
                    for pos in unknown_neighbors: #Se marcan como movimientos seguros las celdas adyacentes
                        safe_moves.add(pos)
    
    #print("Safe moves:", safe_moves)
    return safe_moves

def forward_checking(board, assignments, neighbors_dict): #Verifica si una asignación parcial es consistente con las restricciones del tablero.
    for pos in assignments:
        for nx, ny in neighbors_dict[pos]:
            if board[nx, ny] > 0:
                unknown_neighbors = []
                flagged_neighbors = 0
                for nnx, nny in get_neighbors(nx, ny, board):
                    if board[nnx, nny] == 0:
                        if (nnx, nny) in assignments:
                            if assignments[(nnx, nny)] == 1:
                                flagged_neighbors += 1
                        else:
                            unknown_neighbors.append((nnx, nny))

                if flagged_neighbors > board[nx, ny]: #Si se han marcado más minas de las indicadas en la celda no es una jugada válida
                    return False
                if flagged_neighbors + len(unknown_neighbors) < board[nx, ny]: #Si no se han marcado suficientes minas en las celdas adyacentes no es una jugada válida
                    return False

    return True

def backtrack(board, unassigned_vars, assignments, neighbors_dict): #Backtracking con degree ordering para encontrar una asignación válida de minas en el tablero
    if not unassigned_vars:
        return assignments

    var = max(unassigned_vars, key=lambda v: len(neighbors_dict[v])) #Selecciona la variable con más restricciones
    unassigned_vars.remove(var)

    for value in [0, 1]:  # Intento seguro primero (0), luego mina (1)
        assignments[var] = value
        if forward_checking(board, assignments, neighbors_dict): #Si la asignación es consistente con las restricciones del tablero
            result = backtrack(board, unassigned_vars.copy(), assignments.copy(), neighbors_dict) #Se llama recursivamente con la asignación actual
            if result is not None:
                return result

    unassigned_vars.add(var) 
    return None 

def calculate_risk_with_bt_fc_do(board, known_mines): #Calcula el riesgo de cada celda usando BT-FC-DO
    rows, cols = board.shape
    risk_map = defaultdict(float)
    neighbors_dict = {}

    for x in range(rows):
        for y in range(cols):
            if board[x, y] > 0:
                for nx, ny in get_neighbors(x, y, board):
                    if board[nx, ny] == 0 and (nx, ny) not in known_mines:
                        if (nx, ny) not in neighbors_dict:
                            neighbors_dict[(nx, ny)] = set()
                        neighbors_dict[(nx, ny)].add((x, y))

    unassigned_vars = set(neighbors_dict.keys())
    total_configurations = 0
    mine_counts = defaultdict(int)

    # Se consideran múltiples configuraciones del tablero para calcular el riesgo de cada celda
    for _ in range(100):  # 100 configuraciones aleatorias para mejorar la precisión (se puede aumentar para mejorar)
        assignments = backtrack(board, unassigned_vars.copy(), {}, neighbors_dict) #Se llama al backtracking para encontrar una asignación válida
        if assignments is not None:
            total_configurations += 1
            for pos, value in assignments.items():
                if value == 1:  
                    mine_counts[pos] += 1

    # Se calcula el riesgo de cada celda como la proporción de configuraciones en las que es una mina
    if total_configurations > 0:
        for pos in unassigned_vars:
            risk_map[pos] = mine_counts[pos] / total_configurations

    return risk_map

def find_safe_move(board):
    known_mines = mark_mines(board)
    safe_moves_set = safe_moves(board, known_mines)
    
    if safe_moves_set:
        return safe_moves_set.pop() #Si hay movimientos seguros evidentes, se retorna uno de ellos sin hacer bat-fc-do

    risk_map = calculate_risk_with_bt_fc_do(board, known_mines)
    if risk_map:
        safest_move = min(risk_map, key=risk_map.get)
        if risk_map[safest_move] == 0:  # Si el riesgo es 0, se retorna la celda como una jugada segura
            return safest_move

    return None  # No se encontró una jugada 100% segura

### Algoritmos de detección y construcción de la matriz de juego

Este código implementa un proceso automatizado para capturar la sección mitad izquierda de la pantalla, detectar una cuadrícula en la imagen (correspondiente al juego de buscaminas en la pagina https://www.solitar.io/buscaminas) y convertirla en una matriz de 9x9 utilizando varias técnicas avanzadas de procesamiento de imágenes y OCR (Reconocimiento Óptico de Caracteres).

Para tomar la captura de pantalla se utiliza la biblioteca pyautogui. La captura de pantalla se convierte en un array de numpy y se transforma a formato BGR (el formato de color estándar utilizado por OpenCV) para facilitar su procesamiento posterior. Este paso es crucial para asegurarse de que la imagen de entrada esté en el formato correcto para las operaciones de detección de bordes y reconocimiento de patrones.

Luego, se aplica un filtro de desenfoque gaussiano y la detección de bordes con el algoritmo de Canny utilizando OpenCV. Después de convertir la imagen a escala de grises, se utiliza un filtro gaussiano para suavizarla y reducir el ruido. Posteriormente, se aplica la detección de bordes para identificar los contornos en la imagen. Estos contornos se examinan para encontrar aquellos que tienen una proporción de aspecto cercana a 1:1 (lo que sugiere una forma cuadrada) y un tamaño suficientemente grande para ser considerados como la cuadrícula del juego. Una vez detectada, la cuadrícula se recorta y se ajusta un 3% adicional para eliminar bordes no deseados.

Finalmente, se utiliza la biblioteca easyocr para reconocer los números dentro de cada celda de la cuadrícula. La imagen recortada de la cuadrícula se divide en celdas más pequeñas, cada una correspondiente a una celda del juego. Cada celda se redimensiona para mejorar la precisión del OCR aplicado para identificar números en ellas. Finalmente, se construye una matriz a partir de los números detectados, donde las celdas blancas se representan con un 0, las  azules sin numero con un -1 y las celdas con números detectados se representan con su valor numérico correspondiente.

In [ ]:
#Algoritmo para tomar un screenshot de la región izquierda de la pantalla, detectar la cuadrícula de un juego de buscaminas y convertirlo en una matriz

#Paso 1: Tomar una captura de pantalla de la región izquierda de la pantalla
def take_screenshot(region):
    screenshot = pyautogui.screenshot(region=region)
    screenshot = np.array(screenshot)
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

#Paso 2: Detectar la cuadrícula en la imagen
def detect_grid(image):
    #Convertir la imagen a escala de grises y aplicar un desenfoque gaussiano
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(blurred, 30, 150)

    #Encontrar los contornos en la imagen
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #Iterar sobre los contornos para encontrar la cuadrícula
    for contour in contours:
        #Calcular el perímetro del contorno
        x, y, w, h = cv2.boundingRect(contour)
        
        aspect_ratio = w / float(h)
        if 0.9 <= aspect_ratio <= 1.1 and w > 100 and h > 100:
            #Recortar la cuadrícula detectada
            cropped_image = image[y:y+h, x:x+w]

            #Recortar un 3% adicional de cada borde
            crop_percentage = 0.03
            crop_pixels_y = int(h * crop_percentage)
            crop_pixels_x = int(w * crop_percentage)

            #Aplicar el recorte
            final_cropped_image = cropped_image[crop_pixels_y:h-crop_pixels_y, crop_pixels_x:w-crop_pixels_x]
            
            return final_cropped_image
    
    return None

#Paso 3: Convertir la cuadrícula en una matriz
def grid_to_matrix(grid):
    reader = easyocr.Reader(['en'])
    grid_size = 9
    cell_width = grid.shape[1] // grid_size
    cell_height = grid.shape[0] // grid_size
    matrix = np.zeros((grid_size, grid_size), dtype=int)
    for row in range(grid_size):
        for col in range(grid_size):
            #Calcular las coordenadas de la celda
            x_start = col * cell_width
            y_start = row * cell_height
            x_end = (col + 1) * cell_width
            y_end = (row + 1) * cell_height

            #Recortar la celda y redimensionarla
            cell = grid[y_start:y_end, x_start:x_end]
            resized_cell = cv2.resize(cell, (int(cell.shape[1] * 1.75), (int(cell.shape[0] * 1.75))), interpolation=cv2.INTER_LINEAR)

            #Chequear si la celda es blanca
            mean_color = np.mean(resized_cell)

            if mean_color > 150:  #Si la celda es predominantemente blanca
                matrix[row, col] = 0
            else:  #Si la celda no es blanca
                result = reader.readtext(resized_cell, detail=0, paragraph=False, )
                if result:
                    if result[0] == '8':  #Si el texto detectado es 8, se asume que es un 3 (error común del OCR) 
                        matrix[row, col] = 3
                    else:
                        matrix[row, col] = int(result[0])
                else:
                    matrix[row, col] = -1
    return matrix

### Algoritmo de visualización de la solución

Código para mostrar la jugada segura en el output del programa usando la librería de matplotllib.

In [ ]:
def draw_grid_with_move(matrix, safe_move):
    # Colores para cada número en el tablero
    colors = {
        1: (255, 255, 255),  #Blanco
        2: (0, 255, 0),      #Verde
        3: (0, 0, 255),      #Rojo
        4: (0, 165, 255),    #Naranja
        5: (255, 0, 255),    #Morado
        6: (255, 255, 0),    #Amarillo
        7: (0, 0, 0),        #Negro
    }
    grid_size = matrix.shape[0]
    cell_size = 50  # Tamaño de cada celda en la ilustración
    grid_image = np.ones((grid_size * cell_size, grid_size * cell_size, 3), dtype=np.uint8) * 255  # Imagen en blanco

    for row in range(grid_size):
        for col in range(grid_size):
            x_start = col * cell_size
            y_start = row * cell_size
            x_end = x_start + cell_size
            y_end = y_start + cell_size
            num = matrix[row, col]  

            # Dibujar el rectángulo de la celda
            if num == 0:  # Celda sin descubrir
                cv2.rectangle(grid_image, (x_start, y_start), (x_end, y_end), (0, 0, 0), 2)  # Borde negro, fondo blanco
            else:  # Celda descubierta
                cv2.rectangle(grid_image, (x_start, y_start), (x_end, y_end), (255, 0, 0), -1)  # Fondo azul
                cv2.rectangle(grid_image, (x_start, y_start), (x_end, y_end), (0, 0, 0), 2)  # Borde negro

                if num != -1:  # Si la celda contiene un número
                    cv2.putText(grid_image, str(num), (x_start + 15, y_start + 35), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, colors[num], 2)  # Número blanco

    # Destacar la jugada segura
    safe_row, safe_col = safe_move
    x_safe_start = safe_col * cell_size
    y_safe_start = safe_row * cell_size
    x_safe_end = x_safe_start + cell_size
    y_safe_end = y_safe_start + cell_size

    cv2.rectangle(grid_image, (x_safe_start, y_safe_start), (x_safe_end, y_safe_end), (0, 255, 0), 3)

    return grid_image

### Algoritmo de ejecución del programa

Código para ejecutar el programa principal.

In [ ]:
#Tomar una captura de pantalla de la región izquierda de la pantalla
screen_width, screen_height = pyautogui.size()
region = (0, 0, screen_width // 2, screen_height)
screenshot = take_screenshot(region)

#Detectar la cuadrícula en la imagen
grid = detect_grid(screenshot)

#Convertir la cuadrícula en una matriz
if grid is not None:
    print("Cuadrícula del juego detectada correctamente")
    matrix = grid_to_matrix(grid)
    #Encontrar una jugada segura
    move = find_safe_move(matrix)
    if move is not None:
        print("Jugada segura", move)
        #Mostrar graficamente la jugada segura
        grid_image = draw_grid_with_move(matrix, move)
        plt.figure(figsize=(6, 6))
        plt.imshow(cv2.cvtColor(grid_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
    else:
        print("No se encontró una jugada segura")
else:
    print("No se detecto la cuadrícula del juego en la izquierda de su pantalla")